<a href="https://colab.research.google.com/github/Neu-h/BBG/blob/main/%EB%B9%85%EB%B6%84%EA%B8%B0_%EC%8B%A4%EA%B8%B0_%EC%A0%95%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이상치 처리

scaler() 사용

numpy 활용

In [ ]:
np.where(조건, 참일때 값, 거짓일때 값)

In [ ]:
(t>0.5).sum()    # t(numpy)>0.5 인 값들의 개수
len(t[t>0.5])    # 동일

모델별 parameter

레이블 인코딩

In [ ]:
dfX2['주구매지점'] = dfX2['주구매지점'].astype('category').cat.codes

# 무지성 암기 - 분류편
1) 모듈 import

2) 함수 생성 - 성능 평가, 모델 생성

3) 데이터 불러오기

4) 데이터 전처리 - 데이터 타입 변환, 결측치/이상치 처리

5) 변수간 상관관계

6) 변수 선택, 학습/테스트 데이터 분할

7) 데이터 스플릿, 모델 학습, 성능

8) 최종모델로 재학습, 제출 파일 생성

In [36]:
# 1) 모듈 import
import pandas as pd
import numpy as np
import datetime as dt

pd.options.display.max_rows = 500
pd.options.display.max_columns = 20
pd.set_option('display.float_format', '{:.4f}'.format)     # 이거 헷갈림ㅠ

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression as LR
from sklearn.neighbors import KNeighborsClassifier as KNC
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.ensemble import RandomForestClassifier as RFC
from xgboost import XGBClassifier as XGBC


# 2) 함수 생성 - 성능 평가, 모델 생성, 데이터 생성, 모델 학습

def get_scores(model, xtrain, xtest, ytrain, ytest):
  s1 = model.score(xtrain, ytrain)
  s2 = model.score(xtest, ytest)
  
  ypred = model.predict_proba(xtest)[:, 1]
  s3 = roc_auc_score(ytest, ypred)

  eval = [s1, s2, s3]
  names = 'train_score test_score roc_auc'.split()
  scores = pd.Series(eval, index=names)

  return scores


def make_models(xtrain, xtest, ytrain, ytest):
  temp = pd.DataFrame()

  model_LR = LR().fit(xtrain, ytrain)
  temp['LR'] = get_scores(model_LR, xtrain, xtest, ytrain, ytest)

  for k in range(1,10):
    model_KNC = KNC(k).fit(xtrain, ytrain)
    temp[f'KNC_{k}'] = get_scores(model_KNC, xtrain, xtest, ytrain, ytest)

  model_DTC = DTC(random_state=0).fit(xtrain, ytrain)
  temp['DTC'] = get_scores(model_DTC, xtrain, xtest, ytrain, ytest)

  for d in range(3,9):
    model_DTC = DTC(max_depth=d, random_state=0).fit(xtrain, ytrain)
    temp['DTC'] = get_scores(model_DTC, xtrain, xtest, ytrain, ytest)

  model_RFC = RFC(500, random_state=0).fit(xtrain, ytrain)
  temp['RFC'] = get_scores(model_RFC, xtrain, xtest, ytrain, ytest)

  for n,d in zip([100, 500, 1000, 2000], range(3,9)):
    model_RFC = RFC(n, max_depth=d, random_state=0).fit(xtrain, ytrain)
    temp[f'RFC_{n}_{d}'] = get_scores(model_RFC, xtrain, xtest, ytrain, ytest)

  model_XGBC = XGBC(eval_metric='logloss', use_label_encoder=False).fit(xtrain, ytrain)
  temp['XGBC'] = get_scores(model_XGBC, xtrain, xtest, ytrain, ytest)

  temp = temp.T
  temp.insert(2, 'train-test', (temp['train_score']-temp['test_score']).abs())
  return temp


def make_data(dfXn, Y):
  train_size = len(X_study)
  X_tr = dfXn[:train_size]
  X_te = dfXn[train_size:]
  Y_tr = Y['gender']
  print([a.shape for a in [X_tr, X_te, Y_tr]])      # X끼리는 컬럼 수가 같아야하고, tr끼리는 로우 수가 같아야함!
  return X_tr, X_te, Y_tr


def model_train(dfXn, Y):
  X_tr, X_te, Y_tr = make_data(dfXn, Y)
  
  xtrain, xtest, ytrain, ytest = train_test_split(X_tr, Y_tr,
                                                  test_size=0.3,
                                                  stratify=Y_tr,
                                                  random_state=0)
  result = make_models(xtrain, xtest, ytrain, ytest)
  print(result)
  print('best model: ', np.where(result['roc_auc'] == result['roc_auc'].max()), 'max score: ', result['roc_auc'].max())



In [ ]:
!git clone https://github.com/Soyoung-Yoon/bigdata

In [37]:
# 3) 데이터 가져오기, 전처리 준비
X_study = pd.read_csv('bigdata/X_train.csv', encoding='cp949')
X_exam = pd.read_csv('bigdata/X_test.csv', encoding='cp949')
Y = pd.read_csv('bigdata/y_train.csv', encoding='cp949')
    
dfX = pd.concat([X_study, X_exam], ignore_index=True)
# display(dfX.shape, Y.shape, dfX.head(), Y.head(), dfX.info())
    
    
# 4) 데이터 전처리 - 결측치/이상치 대체, 데이터타입 변환
    
dfX2 = dfX.copy()
dfX2 = dfX2.drop(columns='cust_id')
dfX2['주구매지점'] = dfX2['주구매지점'].astype('category').cat.codes
dfX2['주구매상품'] = dfX2['주구매상품'].astype('category').cat.codes
        
dfX3 = dfX2.copy()
dfX3['환불금액'] = dfX3['환불금액'].mask(dfX3['환불금액'].isna(), 0)
    
dfX4 = dfX2.copy()
a = dfX4.groupby('주구매상품')['환불금액'].transform('mean')
dfX4['환불금액'] = dfX4['환불금액'].mask(dfX4['환불금액'].isna(), a)
b = dfX4['환불금액'].mean()
dfX4['환불금액'] = dfX4['환불금액'].mask(dfX4['환불금액'].isna(), b)
    
dfX5 = dfX2.drop(columns='환불금액')


# 5) 변수간 상관관계 확인

dfXY = pd.merge(dfX, Y)
display(dfXY.corr(), dfX2.info())


for dfXn in [dfX3, dfX4, dfX5]:
  model_train(dfXn, Y)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5982 entries, 0 to 5981
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   총구매액     5982 non-null   int64  
 1   최대구매액    5982 non-null   int64  
 2   환불금액     2076 non-null   float64
 3   주구매상품    5982 non-null   int8   
 4   주구매지점    5982 non-null   int8   
 5   내점일수     5982 non-null   int64  
 6   내점당구매건수  5982 non-null   float64
 7   주말방문비율   5982 non-null   float64
 8   구매주기     5982 non-null   int64  
dtypes: float64(3), int64(4), int8(2)
memory usage: 338.9 KB


,cust_id,총구매액,최대구매액,환불금액,내점일수,내점당구매건수,주말방문비율,구매주기,gender
cust_id,1.0000,-0.0080,-0.0059,0.0701,-0.0252,-0.0048,0.0019,0.0201,-0.0020
총구매액,-0.0080,1.0000,0.7001,0.4197,0.6591,0.0900,0.0144,-0.2129,-0.1501
최대구매액,-0.0059,0.7001,1.0000,0.4106,0.3741,0.0190,0.0223,-0.1158,-0.1143
환불금액,0.0701,0.4197,0.4106,1.0000,0.2703,-0.0631,-0.0624,-0.2111,-0.0780
내점일수,-0.0252,0.6591,0.3741,0.2703,1.0000,0.2253,-0.0103,-0.2932,-0.1550
내점당구매건수,-0.0048,0.0900,0.0190,-0.0631,0.2253,1.0000,0.0077,-0.0912,-0.0439
주말방문비율,0.0019,0.0144,0.0223,-0.0624,-0.0103,0.0077,1.0000,0.0034,0.0736
구매주기,0.0201,-0.2129,-0.1158,-0.2111,-0.2932,-0.0912,0.0034,1.0000,0.0415
gender,-0.0020,-0.1501,-0.1143,-0.0780,-0.1550,-0.0439,0.0736,0.0415,1.0000


None

[(3500, 9), (2482, 9), (3500,)]
            train_score  test_score  train-test  roc_auc
LR               0.6237      0.6238      0.0001   0.6529
KNC_1            1.0000      0.5457      0.4543   0.5228
KNC_2            0.7694      0.6095      0.1599   0.5495
KNC_3            0.7792      0.6010      0.1782   0.5741
KNC_4            0.7212      0.6219      0.0993   0.5884
KNC_5            0.7233      0.6143      0.1090   0.5905
KNC_6            0.7020      0.6381      0.0639   0.5937
KNC_7            0.7049      0.6105      0.0944   0.5902
KNC_8            0.6878      0.6305      0.0573   0.5993
KNC_9            0.6906      0.6143      0.0763   0.5953
DTC              0.7469      0.6238      0.1231   0.6090
RFC              1.0000      0.6495      0.3505   0.6670
RFC_100_3        0.6592      0.6457      0.0135   0.6851
RFC_500_4        0.6755      0.6543      0.0212   0.6898
RFC_1000_5       0.6931      0.6638      0.0293   0.6922
RFC_2000_6       0.7224      0.6657      0.0567   0.6932